In [104]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# coding: utf-8
%matplotlib inline 

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
# pd.options.display.mpl_style = 'default'

In [105]:
import os
# rootdir = '../../hanxiaoyang/AliTraffic/data' #AliTraffic data
rootdir = './'
# os.path.join(rootdir,'gy_contest_link_info.csv')

In [106]:
def extrac_interval(interval):
    return ''.join(interval[11:20].split(':'))
    # 返回 time_interval 中的时间字符。 不带日期信息

In [107]:
def time_feature(combine):
    combine['date_format'] = pd.to_datetime(combine['date'])

#     combine['hour'] = combine['time_interval'].apply(lambda x: int(x[12:14]))
    # partial_data = training_data[training_data.hour.isin([6,7,8])]

    combine['year'] = pd.DatetimeIndex(combine.date_format).year
    combine['month'] = pd.DatetimeIndex(combine.date_format).month#dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    combine['day'] = pd.DatetimeIndex(combine.date_format).day
    combine['dow'] = pd.DatetimeIndex(combine.date_format).dayofweek
    return combine


### 读取筛选后的数据

In [108]:
# time_travel = pd.read_csv('./time_interval_range_6_8.txt')
time_travel = pd.read_csv(os.path.join(rootdir,'time_interval_range_6_8.txt'))
# time_travel['link_ID'] = time_travel['link_ID'].astype(str) 
time_travel = time_feature(time_travel)
time_travel.head()   

,link_ID,date,time_interval,travel_time,interval,hour,date_format,year,month,day,dow
0,3377906284395510514,2016-05-21,"[2016-05-21 07:06:00,2016-05-21 07:08:00)",10.0,07:06:00,7,2016-05-21,2016,5,21,5
1,3377906282418510514,2016-05-21,"[2016-05-21 08:56:00,2016-05-21 08:58:00)",3.5,08:56:00,8,2016-05-21,2016,5,21,5
2,4377906289813600514,2016-05-21,"[2016-05-21 08:00:00,2016-05-21 08:02:00)",2.0,08:00:00,8,2016-05-21,2016,5,21,5
3,4377906289525800514,2016-05-21,"[2016-05-21 08:10:00,2016-05-21 08:12:00)",2.2,08:10:00,8,2016-05-21,2016,5,21,5
4,4377906289525800514,2016-05-21,"[2016-05-21 08:50:00,2016-05-21 08:52:00)",3.2,08:50:00,8,2016-05-21,2016,5,21,5


### ---------------------------------------------------------------------------------------------------------
### 生成测试数据集

In [109]:
# train = time_travel


In [110]:
# 提取2016年6月6点~8点的数据
base_df = time_travel[time_travel['month']==6]

base_df = base_df.sort_values('time_interval')
base_df = base_df.reset_index()
base_df = base_df.drop('index',1)
# base_df
# print base_df['month'].unique()

# 生成测试集
df_test = base_df[(base_df['month']==6)&(base_df['hour']==7)]

def generate_test_data(x):
    to_hour = x[-9:-7]
    if to_hour == '08':
        new_x = x[:12]+'08'+x[14:-9]+'09'+x[-7:]
    else:
        new_x = x[:12]+'08'+x[14:-9]+'08'+x[-7:]
    return new_x
    
df_test['time_interval'] = df_test['time_interval'].apply(generate_test_data)

time_interval = pd.DataFrame({'time_interval':df_test.time_interval.unique()})
# date = pd.DataFrame({'date':df_test.date.unique()})
link_ID = pd.DataFrame({'link_ID':df_test.link_ID.unique()})

time_interval['key']=1
# date['key']=1
link_ID['key']=1

def extract_date_from_interval(interval):
    return interval[1:11]


# time_interval = pd.merge(time_interval, df_test[['date','time_interval']], on='time_interval', how='left')
time_interval['date'] = time_interval['time_interval'].apply(extract_date_from_interval)

pre_test = pd.merge(time_interval,link_ID)
pre_test.info()

# time_interval
pre_test = pre_test.drop('key',1)
pre_test['link_ID'] = pre_test['link_ID'].astype(str)
pre_test['interval'] = pre_test['time_interval'].apply(extrac_interval) 
# base_df.info()

# pre_test['date_format'] = pd.to_datetime(pre_test['date'])

pre_test['hour'] = pre_test['time_interval'].apply(lambda x: int(x[12:14]))
# # partial_data = training_data[training_data.hour.isin([6,7,8])]

# pre_test['year'] = pd.DatetimeIndex(pre_test.date_format).year
# pre_test['month'] = pd.DatetimeIndex(pre_test.date_format).month#dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# pre_test['day'] = pd.DatetimeIndex(pre_test.date_format).day
# # combine['dow'] = pd.DatetimeIndex(combine.date_format).dayofweek
# # base_df['doy'] = pd.DatetimeIndex(base_df.record_date).dayofyear

pre_test = time_feature(pre_test)
pre_test.head()
# test.dtypes


<class 'pandas.core.frame.DataFrame'>
Int64Index: 118800 entries, 0 to 118799
Data columns (total 4 columns):
time_interval    118800 non-null object
key              118800 non-null int64
date             118800 non-null object
link_ID          118800 non-null object
dtypes: int64(1), object(3)
memory usage: 4.5+ MB


,time_interval,date,link_ID,interval,hour,date_format,year,month,day,dow
0,"[2016-06-01 08:00:00,2016-06-01 08:02:00)",2016-06-01,4377906288421600514,080000,8,2016-06-01,2016,6,1,2
1,"[2016-06-01 08:00:00,2016-06-01 08:02:00)",2016-06-01,3377906280395510514,080000,8,2016-06-01,2016,6,1,2
2,"[2016-06-01 08:00:00,2016-06-01 08:02:00)",2016-06-01,4377906285041600514,080000,8,2016-06-01,2016,6,1,2
3,"[2016-06-01 08:00:00,2016-06-01 08:02:00)",2016-06-01,4377906289243600514,080000,8,2016-06-01,2016,6,1,2
4,"[2016-06-01 08:00:00,2016-06-01 08:02:00)",2016-06-01,4377906280784800514,080000,8,2016-06-01,2016,6,1,2


In [111]:
time_travel.head()

,link_ID,date,time_interval,travel_time,interval,hour,date_format,year,month,day,dow
0,3377906284395510514,2016-05-21,"[2016-05-21 07:06:00,2016-05-21 07:08:00)",10.0,07:06:00,7,2016-05-21,2016,5,21,5
1,3377906282418510514,2016-05-21,"[2016-05-21 08:56:00,2016-05-21 08:58:00)",3.5,08:56:00,8,2016-05-21,2016,5,21,5
2,4377906289813600514,2016-05-21,"[2016-05-21 08:00:00,2016-05-21 08:02:00)",2.0,08:00:00,8,2016-05-21,2016,5,21,5
3,4377906289525800514,2016-05-21,"[2016-05-21 08:10:00,2016-05-21 08:12:00)",2.2,08:10:00,8,2016-05-21,2016,5,21,5
4,4377906289525800514,2016-05-21,"[2016-05-21 08:50:00,2016-05-21 08:52:00)",3.2,08:50:00,8,2016-05-21,2016,5,21,5


### 训练数据集 按day hour interval排序

In [112]:
time_travel = time_travel.sort(['month','day','hour','interval'])
time_travel = time_travel.reset_index()
time_travel = time_travel.drop('index',1)
time_travel.head()


,link_ID,date,time_interval,travel_time,interval,hour,date_format,year,month,day,dow
0,4377906280241600514,2016-03-01,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",17.2,06:00:00,6,2016-03-01,2016,3,1,1
1,4377906286681600514,2016-03-01,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1.0,06:00:00,6,2016-03-01,2016,3,1,1
2,4377906289244800514,2016-03-01,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",5.0,06:00:00,6,2016-03-01,2016,3,1,1
3,4377906280344800514,2016-03-01,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",2.0,06:00:00,6,2016-03-01,2016,3,1,1
4,4377906286032600514,2016-03-01,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",16.3,06:00:00,6,2016-03-01,2016,3,1,1


### 组合训练集与测试集

In [113]:
# combine = [time_travel, pre_test]
time_travel['train_flag'] = 1
pre_test['train_flag'] = 0

combine = pd.concat([time_travel, pre_test]) # 测试集没有 travel time列的数据， 值为nan 
combine.head()

,date,date_format,day,dow,hour,interval,link_ID,month,time_interval,train_flag,travel_time,year
0,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280241600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,17.2,2016
1,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286681600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,1.0,2016
2,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906289244800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,5.0,2016
3,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280344800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,2.0,2016
4,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286032600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,16.3,2016


In [114]:
# combine[combine.train_flag==0].values 

In [115]:
time_travel.dtypes

link_ID                  object
date                     object
time_interval            object
travel_time             float64
interval                 object
hour                      int64
date_format      datetime64[ns]
year                      int32
month                     int32
day                       int32
dow                       int32
train_flag                int64
dtype: object

### 读入Line属性表

In [116]:
### ---------------------------------------------------------------------------------------------------------
### 加载 道路属性数据表

In [117]:
# 

link_top_data = pd.read_csv(os.path.join(rootdir,'gy_contest_link_top_20170715.txt'),sep=';')
link_top_data['link_ID'] = link_top_data['link_ID'].astype(str)
link_top_data = link_top_data.fillna('000')

In [118]:
link_info_data = pd.read_csv(os.path.join(rootdir,'gy_contest_link_info.txt'),sep=';')
link_info_data['link_ID'] = link_info_data['link_ID'].astype(str)

In [119]:
# 合并数据
property_data = pd.merge(link_info_data, link_top_data, how='left', on=['link_ID'])

In [120]:
property_data.head()

,link_ID,length,width,link_class,in_links,out_links
0,4377906289869500514,57,3,1,4377906285525800514,4377906281969500514
1,4377906284594800514,247,9,1,4377906284514600514,4377906285594800514
2,4377906289425800514,194,3,1,000,4377906284653600514
3,4377906284525800514,839,3,1,4377906281234600514,4377906280334600514
4,4377906284422600514,55,12,1,3377906289434510514#4377906287959500514,4377906283422600514


### 统计 links numbers 上游Line条数 以及下游Line条数

In [121]:
def agg_count(x):
    if x == '000':
        return 0
    else:
        return len(x.split('#'))
    
# property_data['in_links_nums'] = 1
# property_data['out_links_nums'] = 1
property_data['in_links_nums'] = property_data['in_links'].apply(agg_count)
property_data['out_links_nums'] = property_data['out_links'].apply(agg_count)
# out_nums = property_data[['link_ID','out_links_nums']].groupby('link_ID').agg('sum')

### 合并 merge, line 属性表与通行时间表

In [122]:
# 合并 道路属性数据 与通行时间数据
def data_merge(df):
#     for df in combine:
    df = pd.merge(df, property_data, how='left', on='link_ID')
    return df

# for df in combine:
#     df = data_merge(df)
# # combine = pd.merge(time_travel, property_data, how='left', on=['link_ID'])
# df.head()
combine = pd.merge(combine, property_data, how='left', on='link_ID')

# combine[0].head()

### 提取训练数据构建特征 ： 计算通行速度

In [123]:
# combine

In [124]:
train = combine[combine.train_flag==1]
train['speed'] = train['length']/train['travel_time']

### 节假日特征

In [125]:
def is_holiday(original_data):
    original_data['holiday'] = 0
    original_data.loc[(original_data['date']=='2016-03-08'),['holiday']] = 1
    original_data.loc[(original_data['date']>='2016-04-02') & (original_data['date']<='2016-04-04'),['holiday']] = 1
    original_data.loc[(original_data['date']>='2016-04-30') & (original_data['date']<='2016-05-02'),['holiday']] = 1
    original_data.loc[(original_data['date']>='2016-06-09') & (original_data['date']<='2016-06-11'),['holiday']] = 1
    return original_data
    
combine = is_holiday(combine)

In [126]:
def is_workday(original_data):
    original_data['is_workday'] = 1
    original_data.loc[(original_data['dow']>=5),['is_workday']] = 0
    original_data.loc[(original_data['date']=='2016-06-12'),['is_workday']] = 1
    original_data.loc[(original_data['date']=='2016-04-04'),['is_workday']] = 0
    return original_data

combine = is_workday(combine)

### 预测 speed ，最后换算成时间

### 计算日均通行时长

In [127]:

time_interval_group_data = train[['date','speed']].groupby(['date']).agg(['mean'])
time_interval_group_data.columns = time_interval_group_data.columns.droplevel(0)
time_interval_group_data = time_interval_group_data.reset_index()
# time_interval_group_data.columns = ['time_interval','velocity_mean']
time_interval_group_data = time_interval_group_data.rename(columns = {'mean':'mean_of_date'})
# time_interval_group_data['date'] = pd.to_datetime(time_interval_group_data['date'])
time_interval_group_data.head()

combine = pd.merge(combine, time_interval_group_data, on='date', how='left')
combine.head()

,date,date_format,day,dow,hour,interval,link_ID,month,time_interval,train_flag,...,length,width,link_class,in_links,out_links,in_links_nums,out_links_nums,holiday,is_workday,mean_of_date
0,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280241600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,128,9,1,3377906289044510514,4377906281241600514,1,1,0,1,7.173341
1,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286681600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,7,12,1,4377906285681600514,4377906280329500514,1,1,0,1,7.173341
2,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906289244800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,66,12,1,4377906282532600514,4377906289525800514,1,1,0,1,7.173341
3,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280344800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,19,9,1,4377906289243600514,9377906288175510514,1,1,0,1,7.173341
4,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286032600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,257,12,1,4377906280334600514,000,1,0,0,1,7.173341


### 按照link_id 算均值

In [128]:

time_interval_group_data = train[['link_ID','speed']].groupby(['link_ID']).agg(['mean'])
time_interval_group_data.columns = time_interval_group_data.columns.droplevel(0)
time_interval_group_data = time_interval_group_data.reset_index()
# time_interval_group_data.columns = ['time_interval','velocity_mean']
time_interval_group_data = time_interval_group_data.rename(columns = {'mean':'mean_of_line'})
# time_interval_group_data['date'] = pd.to_datetime(time_interval_group_data['date'])
time_interval_group_data.head()

combine = pd.merge(combine, time_interval_group_data, on='link_ID', how='left')
combine.head()

,date,date_format,day,dow,hour,interval,link_ID,month,time_interval,train_flag,...,width,link_class,in_links,out_links,in_links_nums,out_links_nums,holiday,is_workday,mean_of_date,mean_of_line
0,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280241600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,9,1,3377906289044510514,4377906281241600514,1,1,0,1,7.173341,8.177787
1,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286681600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,12,1,4377906285681600514,4377906280329500514,1,1,0,1,7.173341,5.588512
2,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906289244800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,12,1,4377906282532600514,4377906289525800514,1,1,0,1,7.173341,8.106087
3,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280344800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,9,1,4377906289243600514,9377906288175510514,1,1,0,1,7.173341,9.380149
4,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286032600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,12,1,4377906280334600514,000,1,0,0,1,7.173341,9.060556


### 按照 interval 算均值，

In [ ]:

time_interval_group_data = train[['interval','speed']].groupby(['interval']).agg(['mean'])
time_interval_group_data.columns = time_interval_group_data.columns.droplevel(0)
time_interval_group_data = time_interval_group_data.reset_index()
# time_interval_group_data.columns = ['time_interval','velocity_mean']
time_interval_group_data = time_interval_group_data.rename(columns = {'mean':'mean_of_interval'})
# time_interval_group_data['date'] = pd.to_datetime(time_interval_group_data['date'])
time_interval_group_data.head()

combine = pd.merge(combine, time_interval_group_data, on='interval', how='left')
combine.head()

,date,date_format,day,dow,hour,interval,link_ID,month,time_interval,train_flag,...,link_class,in_links,out_links,in_links_nums,out_links_nums,holiday,is_workday,mean_of_date,mean_of_line,mean_of_interval
0,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280241600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,1,3377906289044510514,4377906281241600514,1,1,0,1,7.173341,8.177787,9.890769
1,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286681600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,1,4377906285681600514,4377906280329500514,1,1,0,1,7.173341,5.588512,9.890769
2,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906289244800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,1,4377906282532600514,4377906289525800514,1,1,0,1,7.173341,8.106087,9.890769
3,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906280344800514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,1,4377906289243600514,9377906288175510514,1,1,0,1,7.173341,9.380149,9.890769
4,2016-03-01,2016-03-01,1,1,6,06:00:00,4377906286032600514,3,"[2016-03-01 06:00:00,2016-03-01 06:02:00)",1,...,1,4377906280334600514,000,1,0,0,1,7.173341,9.060556,9.890769


### 保存数据

In [ ]:
combine.to_csv('train_test_dataset.csv',index=False)